# OcampoATAC2

## Index
1. [Define required clock parameters](#Define-required-clock-parameters)
2. [Download necessary data](#Download-necessary-data)
3. [Load data](#-Load-data)
4. [Extract features and weights](#Extract-features-and-weights)
5. [Load weights into pyaging model](#Load-weights-into-pyaging-model)
6. [Add reference values](#Add-reference-values)
7. [Add preprocessing and postprocesssing steps](#Add-preprocessing-and-postprocesssing-steps)
8. [Check all data objects](#Check-all-data-objects)
9. [Write clock dictionary](#Write-clock-dictionary)
10. [Clear directory](#Clear-directory)

Let's first import some packages:

In [1]:
import os
import marshal
import shutil
import json
import torch
import pandas as pd
import pyaging as pya

## Define required clock parameters

Let's define some required information first:

In [2]:
clock_name = 'ocampoatac2'
data_type = 'atac'
model_class = 'LinearModel'
species = 'Homo sapiens'
year = 2023
approved_by_author = '⌛'
citation = "Morandini, Francesco, et al. \"ATAC-clock: An aging clock based on chromatin accessibility.\" GeroScience (2023): 1-18."
doi = "https://doi.org/10.1007/s11357-023-00986-0"
notes = None

## Download necessary data

#### Download directly with curl

In [3]:
supplementary_url = "https://static-content.springer.com/esm/art%3A10.1007%2Fs11357-023-00986-0/MediaObjects/11357_2023_986_MOESM9_ESM.tsv"
supplementary_file_name = "peaks.tsv"
os.system(f"curl -o {supplementary_file_name} {supplementary_url}")

0

#### Download GitHub repository

In [4]:
github_url = "https://github.com/SunScript0/ATAC-clock.git"
github_folder_name = github_url.split('/')[-1].split('.')[0]
os.system(f"git clone {github_url}")

0

## Load data

#### From CSV file

In [5]:
df = pd.read_table('ATAC-clock/clocks/parallel/2023-03-19_10-06_tpm_all_samples/final_coefs.tsv', index_col=0)
peaks = pd.read_table('peaks.tsv')

## Extract features and weights

First, let's extract the features and weights:

In [6]:
df['feature'] = df.index.tolist()
df['coefficient'] = df['coef']
df = df[df.coef != 0]

df.head()

,mu,var,coef,feature,coefficient
peak1,2.606184,0.048251,0.044979,peak1,0.044979
peak98,1.891547,0.126696,-0.021717,peak98,-0.021717
peak2340,1.576808,0.102339,0.374326,peak2340,0.374326
peak2638,3.113355,0.055103,-0.010966,peak2638,-0.010966
peak2809,1.585774,0.201104,-0.183500,peak2809,-0.183500


Then, let's create lists for features and weights. Be careful about the intercept, as it usually shows up as a feature name.

In [7]:
features = df['feature'][0:-1].tolist()
features = peaks.loc[features, 'desc'].tolist()
weights = torch.tensor(df['coefficient'][0:-1].tolist()).unsqueeze(0)
intercept = torch.tensor([df['coefficient'][-1]])

<ipython-input-7-9e3ed36562a2>:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  intercept = torch.tensor([df['coefficient'][-1]])


## Load weights into pyaging model

#### Linear model

In [8]:
model = pya.models.LinearModel(input_dim=len(features))

model.linear.weight.data = weights.float()
model.linear.bias.data = intercept.float()

model

LinearModel(
  (linear): Linear(in_features=380, out_features=1, bias=True)
)

## Add reference values

Some clocks have reference values in the case of missing features. It is also possible that these values are for preprocessing features rather than the clock features. Let's add a dictionary with the feature names as the keys.

In [9]:
reference_feature_values = None

## Add preprocessing and postprocesssing steps

The preprocessing and postprocessing objects are dictionaries with the following format, with all items required. It takes in x in the form of a numpy array.

In [10]:
def preprocessing_function(x):
    """
    Normalize an array of counts to TPM (Transcripts Per Million) then
    transforms with log1p.
    """
    lengths = preprocessing_helper_objects[0]
    
    # Normalize by length
    tpm = 1000 * (x / lengths)

    # Scale to TPM (Transcripts Per Million)
    tpm = 1e6 * (tpm / tpm.sum(axis=1, keepdims=True))

    tpm_log1p = np.log1p(tpm)

    return tpm_log1p
preprocessing_function_string = marshal.dumps(preprocessing_function.__code__)

def parse_genomic_location(loc: str):
    chrom, positions = loc.split(':')
    start, end = map(int, positions.split('-'))
    length = end - start
    return length
preprocessing_helper_objects = [[parse_genomic_location(peak) for peak in peaks['desc'].tolist()]]

preprocessing = {
    'name': 'tpm_norm_log1p',
    'preprocessing_function': preprocessing_function_string,
    'preprocessing_helper_objects': preprocessing_helper_objects
}

Similarly is the case of postprocessing. Remember that your function must be compatible with torch and is applied to each number individually.

In [11]:
postprocessing = None

## Check all data objects

Let's print all data objects to check if they make sense.

#### features

In [12]:
def my_print_function():
    print(f"There are {len(features)} features.")
    print(features)
pya.utils.print_to_scrollable_output(my_print_function)

#### reference_feature_values

In [13]:
def my_print_function():
    if reference_feature_values:
        print(f"There are {len(reference_feature_values)} reference feature values.")
    print(reference_feature_values)
pya.utils.print_to_scrollable_output(my_print_function)

#### preprocessing

In [14]:
def my_print_function():
    print(preprocessing)
    if preprocessing:
        print(preprocessing_helper_objects)
pya.utils.print_to_scrollable_output(my_print_function)

#### postprocessing

In [15]:
def my_print_function():
    print(postprocessing)
    if postprocessing:
        print(postprocessing)
pya.utils.print_to_scrollable_output(my_print_function)

#### weight_dict

In [16]:
def my_print_function():
    for name, param in model.named_parameters():
        print(f"Layer: {name}")
        print(f"Shape: {param.shape}")
        print(param.data)
pya.utils.print_to_scrollable_output(my_print_function)

## Write clock dictionary

Let's put everything together and save:

In [17]:
clock_dict = {
    # Metadata
    'clock_name': clock_name,
    'data_type': data_type,
    'model_class': model_class,
    'species': species,
    'year': year,
    'approved_by_author': approved_by_author,
    'citation': citation,
    'doi': doi,
    "notes": notes,

    # Data
    'reference_feature_values': reference_feature_values if reference_feature_values else None,
    'preprocessing': preprocessing if preprocessing else None, 
    'features': features,
    'weight_dict': model.state_dict(),
    'postprocessing': postprocessing if postprocessing else None,
}

torch.save(clock_dict, f'../weights/{clock_name}.pt')

## Clear directory

Delete all files that are not clock jupyter notebooks:

In [18]:
# Function to remove a folder and all its contents
def remove_folder(path):
    try:
        shutil.rmtree(path)
        print(f"Deleted folder: {path}")
    except Exception as e:
        print(f"Error deleting folder {path}: {e}")

# Get a list of all files and folders in the current directory
all_items = os.listdir('.')

# Loop through the items
for item in all_items:
    # Check if it's a file and does not end with .ipynb
    if os.path.isfile(item) and not item.endswith('.ipynb'):
        os.remove(item)
        print(f"Deleted file: {item}")
    # Check if it's a folder
    elif os.path.isdir(item):
        remove_folder(item)

Deleted file: peaks.tsv
Deleted folder: .ipynb_checkpoints
Deleted folder: ATAC-clock
